# 対抗馬手法　kmeans++
    kmeans++アルゴリズムで各FPGAの周波数をクラスタリング

- XPの論文では，経年劣化したFPGAは周波数の分布が異なると予測
- 実際にHSPICEを用いたシュミレーションでも，経年劣化したFPGAは周波数が低い&多峰性
- クラスタ数が他と異なるところを異常値として判定する

# 手法
    kmeans++  シルエット値

## [kmeans++]
1. 中心点の初期値を確率的に決定
2. データポイントと中心点とのユークリッド平方距離を計算
3. 中心点との距離が最も近いクラスターに分類
4. すべての点(クラスター内)の平均を計算して，中心点を割り当て
5. 再び，新しい中心点とのユークリッド平方距離を求め，距離が近いクラスターに分類
6. 3,4,5を繰り返し，中心点の位置を移動

### 中心点の初期値を確率的に決定
1. データ点のうち一つをランダムに選択し，その座標をクラスタ1の中心点の初期値とする
2. 入力される全てのデータ点と，すでに座標が決定している中心点のうち最も近いものとのユークリッド距離を計算
3. いずれかのデータ点を次の中心点とする，
    - その際のデータ点X(p)の選ばれる確率
    $$
    \dfrac {d\left( x_{p}\right) ^{2}}{\sum ^{n-1}_{i=0}d\left( x_{i}\right) ^{2}}
    $$
    - 最近某クラスタから遠ければ遠いほど，選ばれる確率は高くなる
    - すでに中心点として選ばれているデータ点との距離は0なので，分母の総和にも影響を与えないし，選ばれる確率も0となる
4. 手順2-3をk個の中心点が決定するまで繰り返す

- kmeans++を使うことで，計算速度(実行速度)が上がり，適切にクラスタできる確率も上がる
- 極値も避けることができる

### kmeansの問題点
- kmeansアルゴリズムはクラスタ数を人の手で決定しなければならない

## [シルエット値]
    シルエット値を用いることで，適切なクラスタ数を定量的に決定することができる．

- シルエット値の計算
- あるデータ点と同クラスター内のデータ店とのユークリッド平方距離の平均　OCI
- あるデータ点と近隣クラスター内のデータ点とのユークリッド平方距離の平均　NCI

$$
\dfrac {NC_{i}-OC_{i}}{\max \left\{ NC_{i},OC_{i}\right\}}
$$

- 1に近いほど，よく当てはまってる

### 論文では

#### 手順
1. それぞれのデータ点のシルエット値を計算する
2. それぞれのクラスター数でのシルエット値の平均を出す
3. 最も高いシルエット平均を出したクラスター数を採用

#### その他

- ACNをTCN（閾値）で区別する
- 閾値は，使っていないFPGAのACNに近い値にする（平均？）
- 論文では決め方を近い値としか書いてなかった

- 論文内では，三つのLUTでの検出率を調べていた
- また，経年劣化日数での検出率の差を示していた（1day, 1week, 1month, 6month)

- FPGAの種類はSparten 3E だった
- このFPGAではACNの閾値を3以下を正常，3より上を異常と判断すると良いという結論